# nb_03_modelagem_gold

Objetivo: Construção do modelo analítico na camada Gold, utilizando modelagem dimensional em esquema estrela a partir dos dados tratados da camada Silver.

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS gold;


In [0]:
%sql
USE CATALOG mvp_catalog;
USE SCHEMA silver;


In [0]:
%sql
DROP TABLE IF EXISTS gold.dim_product;

CREATE TABLE gold.dim_product
USING DELTA
AS
SELECT
  p.product_id,
  p.product_name,
  p.product_number,
  p.standard_cost,
  p.list_price,
  p.sell_start_date,
  p.sell_end_date,

  sc.product_subcategory_id,
  sc.subcategory_name,

  c.product_category_id,
  c.category_name

FROM mvp_catalog.silver.product p
LEFT JOIN mvp_catalog.silver.product_subcategory sc
  ON p.product_subcategory_id = sc.product_subcategory_id
LEFT JOIN mvp_catalog.silver.product_category c
  ON sc.product_category_id = c.product_category_id;



In [0]:
%sql
DROP TABLE IF EXISTS gold.dim_customer;

CREATE TABLE gold.dim_customer
USING DELTA
AS
SELECT
  customer_id,
  first_name,
  last_name,
  person_type,
  email_address
FROM mvp_catalog.silver.customer;


In [0]:
%sql
DROP TABLE IF EXISTS gold.dim_territory;

CREATE TABLE gold.dim_territory
USING DELTA
AS
SELECT
  territory_id,
  territory_name,
  country_region_code,
  territory_group
FROM mvp_catalog.silver.sales_territory;


In [0]:
%sql
DROP TABLE IF EXISTS gold.dim_date;

CREATE TABLE gold.dim_date
USING DELTA
AS
SELECT DISTINCT
  CAST(order_date AS DATE)      AS date,
  year(order_date)              AS year,
  month(order_date)             AS month,
  date_format(order_date,'MMMM') AS month_name,
  day(order_date)               AS day
FROM mvp_catalog.silver.sales;


In [0]:
%sql
DROP TABLE IF EXISTS gold.fact_sales;

CREATE TABLE gold.fact_sales
USING DELTA
AS
SELECT
  sales_order_id,
  sales_order_detail_id,
  CAST(order_date AS DATE) AS order_date,
  customer_id,
  product_id,
  territory_id,
  order_qty,
  unit_price,
  line_total
FROM mvp_catalog.silver.sales;


In [0]:
%sql
SHOW TABLES IN gold;


In [0]:
%sql
SELECT
  SUM(f.line_total) AS receita_total
FROM gold.fact_sales f
JOIN gold.dim_product p
  ON f.product_id = p.product_id;


In [0]:
%sql
SELECT *
FROM gold.fact_sales
LIMIT 10;


In [0]:
%sql
DESCRIBE TABLE mvp_catalog.gold.fact_sales

In [0]:
%sql
DESCRIBE TABLE mvp_catalog.gold.dim_product